# BGE-M3 应用

In [1]:
# 1. Dense Embedding 密集嵌入

from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel(model_name_or_path="/slurm/resources/weights/huggingface/BAAI/bge-m3",
                       use_fp16=True)

sentences_1 = ["这个周末我计划去海边度假，享受阳光和海浪", "最新研究表明，定期运动有助于提高工作效率和创造力"]
sentences_2 = ["我期待的假期是在沙滩上，听着海浪声放松", "科技公司最近发布了一款新的智能手机，引起了广泛关注"]

embeddings_1 = model.encode(
    sentences_1,
    batch_size=12,
    max_length=1024,
)["dense_vecs"]

embeddings_2 = model.encode(
    sentences_2,
    batch_size=12,
    max_length=1024,
)["dense_vecs"]

similarity = embeddings_1 @ embeddings_2.T

print(similarity)

/slurm/home/admin/.conda/envs/dl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[[0.786  0.4346]
 [0.4368 0.4753]]


In [ ]:
# 2. Sparse Embedding (Lexical Weight) 稀疏嵌入（词汇权重）

from FlagEmbedding import BGEM3FlagModel
from pprint import pprint

model = BGEM3FlagModel(model_name_or_path="/slurm/resources/weights/huggingface/BAAI/bge-m3",
                       use_fp16=True)

sentences_1 = ["这个周末我计划去海边度假，享受阳光和海浪", "最新研究表明，定期运动有助于提高工作效率和创造力"]
sentences_2 = ["周末去海边，听着海浪声方式，享受阳光", "科技公司最近发布了一款新的智能手机，引起了广泛关注"]

output_1 = model.encode(sentences_1, return_dense=True, return_sparse=True, return_colbert_vecs=False)

output_2 = model.encode(sentences_2, return_dense=True, return_sparse=True, return_colbert_vecs=False)

# 查看每个token的权重
pprint(model.convert_id_to_token(output_1["lexical_weights"]))

In [ ]:
# 通过词汇匹配计算得分

lexical_scores = model.compute_lexical_matching_score(output_1["lexical_weights"][0],
                                                      output_2["lexical_weights"][0])

print(lexical_scores)

In [ ]:
print(model.compute_lexical_matching_score(output_1["lexical_weights"][0],
                                           output_2["lexical_weights"][1]))

In [ ]:
# 3. Multi-Vector (ColBERT) 多向量

from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel("/slurm/resources/weights/huggingface/BAAI/bge-m3", use_fp16=True)

sentences_1 = ["这个周末我计划去海边度假，享受阳光和海浪", "最新研究表明，定期运动有助于提高工作效率和创造力"]
sentences_2 = ["周末去海边，听着海浪声方式，享受阳光", "科技公司最近发布了一款新的智能手机，引起了广泛关注"]

output_1 = model.encode(sentences_1, return_dense=True, return_sparse=True, return_colbert_vecs=True)

output_2 = model.encode(sentences_2, return_dense=True, return_sparse=True, return_colbert_vecs=True)

print(model.colbert_score(output_1["colbert_vecs"][0], output_2["colbert_vecs"][0]))

In [ ]:
print(model.colbert_score(output_1["colbert_vecs"][0], output_2["colbert_vecs"][1]))

In [ ]:
# 4. 计算文本对的分数 
# 输入文本对列表，即可得到不同方法计算出的分数。

sentences_1 = ["什么是BGE M3？", "BM25的定义"]
sentences_2 = ["BGE M3是一个支持密集检索、词汇匹配和多向量交互的嵌入模型。", 
               "BM25是一种词袋检索函数，它根据查询词汇在每个文档中出现的情况对一组文档进行排名"]

# 构建文本对
sentence_pairs = [[i,j] for i in sentences_1 for j in sentences_2]

print(sentence_pairs)

In [ ]:
pprint(model.compute_score(sentence_pairs,
                          max_passage_length=128,
                          # weights_for_different_modes(w) 用于执行加权和：
                          # w[0]*dense_score + w[1]*sparse_score + w[2]*colbert_score
                          weights_for_different_modes=[0.4, 0.2, 0.4]))

## BGE-M3 微调
训练数据应该是一个 jsonl 文件，其中每一行都是一个像这样的字典：  
```shell
{"query": str, "pos": List[str], "neg":List[str]}
```  

如果你想使用知识蒸馏，你的 jsonl 文件的每一行应该是这样的：  
```shell
{"query": str, "pos": List[str], "neg":List[str], "pos_scores": List[float], "neg_scores": List[float]}
```   

`pos_scores`是正分数列表，其中`pos_scores[i]`是查询与教师模型中的`pos[i]`之间的分数。     
`neg_scores`是负分数列表，其中`neg_scores[i]`是查询与教师模型中的`neg[i]`之间的分数。   



下面是一个简单的例子，说明如何基于BAAI/bge-m3进行统一微调（密集嵌入、稀疏嵌入和colbert）：  

```shell
# run_BGM-M3_finetune.sh

torchrun --nproc_per_node 1 \
-m FlagEmbedding.BGE_M3.run \
--output_dir /root/autodl-tmp/flagembedding/output_M3_finetuned_model \
--model_name_or_path /root/autodl-tmp/models/bge-m3 \
--train_data /root/autodl-tmp/samples/ \
--learning_rate 1e-5 \
--fp16 \
--num_train_epochs 1 \
--per_device_train_batch_size 1 \
--dataloader_drop_last True \
--normlized True \
--temperature 0.02 \
--query_max_len 64 \
--passage_max_len 256 \
--train_group_size 2 \
--negatives_cross_device \
--logging_steps 10 \
--same_task_within_batch True \
--unified_finetuning True \
--use_self_distill True
```  